In [1]:
from datetime import datetime
import lxml
from lxml import html
import requests
import numpy as np
import pandas as pd

In [3]:
symbol = 'EGIE3.SA'

https://finance.yahoo.com/quote/EGIE3.SA/balance-sheet?p=EGIE3.SA

In [7]:
url = 'https://finance.yahoo.com/quote/' + symbol + '/balance-sheet?p=' + symbol

In [8]:
# Set up the request headers that we're going to use, to simulate
# a request by the Chrome browser. Simulating a request from a browser
# is generally good practice when building a scraper
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'max-age=0',
    'Pragma': 'no-cache',
    'Referrer': 'https://google.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'
}

# Fetch the page that we're going to parse, using the request headers
# defined above
page = requests.get(url, headers)

# Parse the page with LXML, so that we can start doing some XPATH queries
# to extract the data that we want
tree = html.fromstring(page.content)

# Smoke test that we fetched the page by fetching and displaying the H1 element
tree.xpath("//h1/text()")

['EGIE3.SA - Engie Brasil Energia S.A.']

### reading the Financial data

In [19]:
table_rows = tree.xpath("//div[contains(@class, 'D(tbr)')]")
assert len(table_rows) >0

[<Element div at 0x24c0035c368>,
 <Element div at 0x24c00456368>,
 <Element div at 0x24c00456a48>,
 <Element div at 0x24c00456b38>,
 <Element div at 0x24c00456b88>,
 <Element div at 0x24c00456bd8>,
 <Element div at 0x24c00456c28>,
 <Element div at 0x24c00456c78>,
 <Element div at 0x24c00456cc8>,
 <Element div at 0x24c00456d18>,
 <Element div at 0x24c00456d68>,
 <Element div at 0x24c00456db8>,
 <Element div at 0x24c00456e08>,
 <Element div at 0x24c00456e58>,
 <Element div at 0x24c00456ea8>]

In [13]:
parsed_rows = []

In [18]:
for table_row in table_rows:
    parsed_row = []
    el = table_row.xpath('./div')
    none_count = 0
    
    for rs in el:
        try:
            (text,)=  rs.xpath('.//span/text()[1]')
            parsed_row.append(text)
        except ValueError:
            parsed_row.append(np.NaN)
            none_count +=1
    if( none_count < 4):
        parsed_rows.append(parsed_row)
df = pd.DataFrame(parsed_rows)        
df

,0,1,2,3,4
0,Breakdown,12/31/2019,12/31/2018,12/31/2017,12/31/2016
1,Total Assets,"30,135,578","23,735,545","19,568,513","14,419,691"
2,Total Liabilities Net Minority Interest,"23,136,758","17,414,968","12,733,788","7,805,297"
3,Total Equity Gross Minority Interest,"6,998,820","6,320,577","6,834,725","6,614,394"
4,Total Capitalization,"18,916,052","15,371,538","10,511,092","9,400,216"
5,Common Stock Equity,"6,995,154","6,316,186","6,830,594","6,611,227"
6,Capital Lease Obligations,"134,307",NaN,NaN,NaN
7,Net Tangible Assets,"5,698,385","5,003,341","5,521,643","6,358,713"
8,Working Capital,"766,027","386,416","-1,940,692","1,554,297"
9,Invested Capital,"21,758,212","16,036,420","13,587,010","9,699,959"


In [1]:
#Finding a way to scrappe all lines, theres a problem because the page don't render everything (js action needed)